<a href="https://colab.research.google.com/github/parthag1201/Authentication_google/blob/main/FS_TS_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages for crew and langchain
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [ ]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process

In [ ]:
import os
from google.colab import userdata # For API Secret
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

os.environ["GEMINI_MODEL_NAME"] = 'gemini-2.5-pro'

In [ ]:
abap_code_string = """
REPORT Z_MCCAIN_SALES_ANALYSIS.

TABLES: vbak, vbap, mara.

DATA: gt_sales_data TYPE TABLE OF ty_sales,
      gs_sales_data TYPE ty_sales.

*----------------------------------------------------------------------*
* Selection Screen
*----------------------------------------------------------------------*
SELECT-OPTIONS: s_vbeln FOR vbak-vbeln,
                s_matnr FOR mara.matnr.
PARAMETERS:     p_top TYPE i DEFAULT 10.

*----------------------------------------------------------------------*
* Start of Selection
*----------------------------------------------------------------------*
START-OF-SELECTION.
  PERFORM get_sales_data.
  PERFORM display_sales_data.

*&---------------------------------------------------------------------*
*&      Form  GET_SALES_DATA
*&---------------------------------------------------------------------*
FORM get_sales_data.
  SELECT vbeln, posnr, matnr, netwr
    FROM vbap
    INTO CORRESPONDING FIELDS OF TABLE gt_sales_data
    WHERE vbeln IN s_vbeln AND matnr IN s_matnr
    UP TO p_top ROWS.
ENDFORM.

*&---------------------------------------------------------------------*
*&      Form  DISPLAY_SALES_DATA
*&---------------------------------------------------------------------*
FORM display_sales_data.
  LOOP AT gt_sales_data INTO gs_sales_data.
    WRITE: / gs_sales_data-vbeln,
             gs_sales_data-posnr,
             gs_sales_data-matnr,
             gs_sales_data-netwr.
  ENDLOOP.
ENDFORM.
"""

In [ ]:
template_TS = """Table of Contents

1. Enhancement Narrative

1.1 Request Details

1.2 Enhancement Definition

2. Enhancement Functional Requirements

2.1 Business Requirements

2.1.1 Objective

2.1.2 Scope

2.2 As-Is/To-Be Functionality

2.2.1 Special Requirements

2.3 SAP Standard Reference

2.4 Inputs and Dependencies

2.5 Regulatory / Compliance Requirements

2.6 Assumptions

3. Enhancement Specifications

3.1 User Exit /BADI/Routines/ Details

3.2 Transaction Screens

3.2.1 Screen Flow

3.3 Enhancement Processing Logic – Key steps

3.4 Table Data Access

4. Error Handling

5. Level 0 Test Requirements

6. Additional Information

7. Acronyms, Abbreviations, and Terms

8. References

9. Document Attributes"""

In [ ]:
template_FS = """
1. Enhancement Narrative

1.1 Request Details

1.2 Enhancement Definition

2. Enhancement Functional Requirements

2.1 Business Requirements

2.1.1 Objective

2.1.2 Scope

2.2 As-Is/To-Be Functionality

2.2.1 Special Requirements

2.3 SAP Standard Reference

2.4 Inputs and Dependencies

2.5 Regulatory / Compliance Requirements

2.6 Assumptions

3. Enhancement Specifications

3.1 User Exit /BADI/Routines/ Details

3.2 Transaction Screens

3.2.1 Screen Flow

3.3 Enhancement Processing Logic – Key steps

3.4 Table Data Access

4. Error Handling

5. Level 0 Test Requirements

6. Additional Information

7. Acronyms, Abbreviations, and Terms

8. References

9. Document Attributes
"""

In [ ]:
# Agents
from langchain_community.llms import HuggingFaceHub # Import necessary class if using Hugging Face models
from langchain_google_genai import ChatGoogleGenerativeAI # Import the correct class for Gemini

# Initialize the Gemini LLM
os.environ['GOOGLE_API_KEY']=userdata.get('Gemini_ai_api')
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


abap_code_analyst = Agent(
    role="Senior ABAP Code Analyst",
    goal="Dissect the provided ABAP report code to extract all technical details and the logic flow of the report."
    "Focus on identifying data sources, selection screens, core logic, and output structures."
    "You need to go through each and every section of report to the extent form where a good Functional and Technical specification can be derived out of it"
    "This is the ABAP report code : '{abap_report_code}'",
    backstory="""You are an expert ABAP developer with decades of experience. You have an exceptional eye for detail
    and can instantly understand the flow and structure of any ABAP program. Your task is to analyze the
    code and provide a structured, raw analysis for the documentation team.""",
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm,
)

functional_spec_drafter = Agent(
    role="Business Systems Analyst",
    goal="Translate the technical analysis of an ABAP report into a clear and concise"
   "functional specification document for business stakeholders."
   "Refer to this professional template for creating a business functional specification document. '{template_FS}'",
    backstory="""You are a bridge between the technical team and the business. You excel at taking complex
    technical jargon and reframing it in terms of business processes and objectives. Your functional
    specifications are legendary for their clarity and business relevance, enabling stakeholders to
    understand exactly what a report does.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

technical_spec_writer = Agent(
    role="Technical Documentation Specialist",
    goal="""Create a comprehensive and detailed technical specification document based on the
    ABAP code analysis."""
    "Refer to this professional template for creating a business technical specification document. '{template_TS}'",
    backstory="""You are a meticulous technical writer who specializes in creating documentation for developers.
    Your specifications are precise, well-structured, and follow industry best practices. Your work ensures
    that any developer can pick up the report for maintenance or enhancement with complete clarity
    on its internal workings.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

manager_agent = Agent(
    role='Manager Agent',
    goal='Oversee the generation and refinement of Functional and Technical Specifications, facilitate collaboration, incorporate reviewer feedback, and ensure final output meets all requirements.'
    'The FS and TS reports must contain all the important information extracted from the report fror the business team to understand each and every ascpet of the report easily',
    backstory="""You are an experienced SAP S4 project manager with a deep understanding of software development life cycles and well versed with the good practises of SAP, Busness needs and ABAP Development.
    You excel at breaking down complex tasks, coordinating teams, incorporating feedback, and ensuring quality deliverables.
    You are adept at communicating with both technical and business stakeholders.""",
    verbose=True,
    allow_delegation=True,
    llm=gemini_llm,
)

output_reviewer = Agent(
    role='Output Reviewer (Business Knowledge)',
    goal='Review consolidated Functional and Technical Specifications and provide constructive feedback to the Manager Agent for refinement, focusing on business alignment and completeness.',
    backstory="""You are a business stakeholder with a keen eye for detail and a deep understanding of the organizational goals and user needs.
    You critically evaluate specifications, ensuring they meet intended business objectives and provide actionable feedback for improvement.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
)

In [ ]:
# Tasks

analyze_code_task = Task(
    description="""Analyze the following ABAP report code: '{abap_report_code}'.
                   Your analysis must cover these key areas:
                   1.  **Data Sources:** Identify all database tables, views, or structures being used.
                   2.  **Selection Screen:** Detail all SELECT-OPTIONS and PARAMETERS, including their technical IDs and associated data elements.
                   3.  **Core Processing Logic:** Summarize the main logic blocks, including loops, conditionals (IF/CASE statements), and subroutine calls (PERFORM).
                   4.  **Data Output:** Describe how the final data is presented (e.g., ALV grid, classical report WRITE statements) and list the fields being displayed.""",
    expected_output="""A structured text document containing a raw, point-by-point technical breakdown of the ABAP report.
                       This output should be purely technical and serve as the foundational information for other agents.""",
    agent=abap_code_analyst,
)

# Task for the Functional Spec Drafter
functional_spec_task = Task(
    description="""Using the technical analysis provided by the ABAP Code Analyst, create a functional specification document.
                   The document should explain the report's purpose and functionality from a business user's perspective.
                   It must include:
                   1.  **Report Purpose:** A high-level summary of what the report achieves.
                   2.  **Selection Criteria:** Explain the input fields in simple business terms (e.g., "User can filter by Sales Document Number").
                   3.  **Processing Logic Summary:** Describe what the report does with the data in plain language (e.g., "The report calculates the total net value for the selected sales orders").
                   4.  **Output Description:** Describe the layout and columns of the final report from a user's point of view.""",
    expected_output="""A well-formatted functional specification document in Markdown. The language should be clear,
                       non-technical, and focused on the business value and utility of the report.""",
    agent=functional_spec_drafter,
)

# Task for the Technical Spec Writer
technical_spec_task = Task(
    description="""Using the detailed technical analysis from the ABAP Code Analyst, create a formal technical specification document.
                   This document must be structured for a developer audience and include:
                   1.  **Program Details:** Program ID, Title, and a brief technical overview.
                   2.  **Data Declarations:** List of key tables, internal tables, and complex variables.
                   3.  **Selection Screen Objects:** A table listing each selection screen field with its technical name and properties.
                   4.  **Detailed Logic Flow:** A step-by-step description of the program's execution logic, referencing specific subroutines and key logic blocks.
                   5.  **Output Details:** A technical breakdown of the output display, including the field catalog for ALV grids or the format of WRITE statements.""",
    expected_output="""A comprehensive technical specification document in Markdown. It should be precise, detailed, and
                       formatted professionally to serve as official technical documentation.""",
    agent=technical_spec_writer,
)


# Manager Task for initial consolidation
initial_consolidation_task = Task(
    description="""Consolidate the initial Functional and Technical Specifications into a single, cohesive document.
    Ensure both specifications are present and well-formatted for review.
    """,
    expected_output="A single Markdown document containing the initial Functional Specification and Technical Specification, clearly separated.",
    agent=manager_agent,
    context=[functional_spec_task, technical_spec_task],
    llm=gemini_llm,
)

# Output Reviewer Task to provide feedback to the manager
output_review_feedback_task = Task(
    description="""Review the provided consolidated Functional and Technical Specifications.
    Focus on business alignment, clarity, completeness, and accuracy from a business perspective.
    Provide constructive feedback to the Manager Agent, detailing any required changes or improvements.
    The output should be a clear, concise, and structured list of actionable feedback points, not a final report.
    """,
    expected_output="A structured list of feedback points (e.g., bullet points, numbered list) for the Manager Agent to use for refining the specifications. Example: '- Section X needs more detail on Y. - Clarify business impact of Z. - Ensure consistency between FS and TS for ABC.'",
    agent=output_reviewer,
    context=[initial_consolidation_task], # Reviewer reviews the initial consolidated output
    llm=gemini_llm,
)

# Manager Task for final refinement based on feedback
final_specification_task = Task(
    description="""Based on the initial Functional and Technical Specifications and the feedback provided by the Output Reviewer,
    make necessary revisions to both specifications. Your goal is to produce the final, polished, and
    approved Functional and Technical Specifications in a single consolidated document.
    The output must contain the actual Functional Specification and Technical Specification,
    incorporating all valid feedback. Ensure clear separation between the FS and TS sections in the final output.
    """,
    expected_output="A single, comprehensive Markdown document containing the final, refined Functional Specification and Technical Specification, clearly separated (e.g., using distinct headings like '# Functional Specification' and '# Technical Specification') and incorporating all reviewer feedback.",
    agent=manager_agent,
    context=[initial_consolidation_task, output_review_feedback_task], # Manager uses both initial specs and feedback
    llm=gemini_llm,
    output_file='final_consolidated_specifications.md' # Final output will be saved here
)

In [ ]:
# Call the crew

abap_doc_crew = Crew(
    agents=[
        abap_code_analyst,
        functional_spec_drafter,
        technical_spec_writer,
        manager_agent,
        output_reviewer
    ],
    tasks=[
        analyze_code_task,
        functional_spec_task,
        technical_spec_task,
        initial_consolidation_task,
        output_review_feedback_task,
        final_specification_task # This task produces the final output after review
    ],
    process=Process.sequential,
    verbose=2
)

In [ ]:
# Start execution

result = abap_doc_crew.kickoff(
    inputs={
        'abap_report_code': abap_code_string,
        'template_FS' : template_FS,
        'template_TS' : template_TS
        })

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Consolidated Report Specifications: Z_MCCAIN_SALES_ANALYSIS

---

# Functional Specification Document: Z_MCCAIN_SALES_ANALYSIS

## 1. Enhancement Narrative

### 1.1 Request Details
This document outlines the functional requirements for a new custom ABAP report, `Z_MCCAIN_SALES_ANALYSIS`, which aims to provide quick and focused insights into sales order item details. The request stems from a business need for a streamlined view of sales item data, allowing for specific filtering and a controlled output size.

### 1.2 Enhancement Definition
The `Z_MCCAIN_SALES_ANALYSIS` report, officially titled **"McCain Sales Order Item Analysis Report"**, is designed to extract and display key sales order item information. It will allow users to query sales data based on specific sales documents or materials and limit the output to a 'Top N' number of records based on defined business criteria. This report will serve as a quick lookup tool for sales item details, including quantity and unit of measure, which were identified as critical for comprehensive analysis.

## 2. Enhancement Functional Requirements

### 2.1 Business Requirements

#### 2.1.1 Objective
The primary objective of the "McCain Sales Order Item Analysis Report" is to support quick lookup and focused review of sales item details, including quantity and unit of measure, for specific sales documents or materials. This report is intended to provide immediate, actionable insights into sales item performance without requiring extensive aggregation or drill-down capabilities in this initial release.

#### 2.1.2 Scope
The report will display the following key sales item information for selected sales orders:
*   Sales Document Number
*   Sales Item Number
*   Material Number
*   Net Value
*   **Currency Unit (from Sales Document Header)**
*   **Quantity (VBAP-KWMENG)**
*   **Unit of Measure (VBAP-VRKME)**

Users will be able to filter the data using the following input parameters on the selection screen:
*   Sales Document Number (single value or range)
*   Material Number (single value or range)
*   Top N Rows (numeric value to limit output)

The report will present the data in a clear, tabular format.

### 2.2 As-Is/To-Be Functionality

**As-Is:** Currently, business users rely on standard SAP transactions (e.g., VA03) for individual sales order lookups or more complex, broader reports that may not offer the targeted, quick overview required for specific sales item analysis. There is no dedicated, lightweight report for this specific purpose.

**To-Be:** The "McCain Sales Order Item Analysis Report" (`Z_MCCAIN_SALES_ANALYSIS`) will fill this gap by providing a user-friendly, focused report that allows for efficient retrieval and review of critical sales item data, including quantity and unit of measure, with the added benefit of a 'Top N Rows' limit for managing output size.

#### 2.2.1 Special Requirements
The 'Top N Rows' functionality requires a precise and consistent ordering logic to ensure that the "top" records are always determined by business-relevant criteria. This logic is crucial for the report's utility and reliability.

### 2.3 SAP Standard Reference
This custom report will primarily reference data from standard SAP Sales and Distribution (SD) tables, specifically:
*   Sales Document Header: `VBAK`
*   Sales Document Item: `VBAP`

### 2.4 Inputs and Dependencies
*   **Inputs:** User-provided values for Sales Document Number, Material Number, and the 'Top N Rows' limit on the report's selection screen.
*   **Dependencies:** The report is dependent on the accurate and timely creation and update of sales orders within the SAP system. Data consistency in `VBAK` and `VBAP` tables is assumed.

### 2.5 Regulatory / Compliance Requirements
No specific regulatory or compliance requirements beyond standard data privacy and access controls applicable to sales data within SAP.

### 2.6 Assumptions
*   Users have the necessary authorizations to execute custom ABAP reports and access sales order data.
*   The required sales order item data (Sales Document, Item, Material, Net Value, Quantity, Unit of Measure) is consistently maintained in the SAP system.
*   The report will be executed primarily for ad-hoc analysis and quick lookups, rather than for large-scale data extracts or complex analytical processing.

## 3. Enhancement Specifications

### 3.1 User Exit /BADI/Routines/ Details
Not applicable, as this is a new standalone custom report (Z-report) developed from scratch, not an enhancement to existing standard SAP functionality via User Exits, BADIs, or standard routines.

### 3.2 Transaction Screens

#### 3.2.1 Screen Flow
1.  **Selection Screen (Initial Screen):**
    *   **Program Title:** McCain Sales Order Item Analysis Report
    *   **Input Fields:**
        *   Sales Document: Input field (e.g., `VBELN` from `VBAK`), allows single value, range, or multiple selections.
        *   Material Number: Input field (e.g., `MATNR` from `VBAP`), allows single value, range, or multiple selections.
        *   Top N Rows: Numeric input field, default value can be defined (e.g., 100), accepts positive integers.
    *   **Execution:** User clicks the "Execute" button (F8).

2.  **Output List (ALV Grid Display):**
    *   Upon execution, the report will display the selected sales item data in an SAP List Viewer (ALV) grid format.
    *   **Columns Displayed:**
        *   Sales Document Number (`VBAK-VBELN`)
        *   Sales Item Number (`VBAP-POSNR`)
        *   Material Number (`VBAP-MATNR`)
        *   Net Value (`VBAP-NETWR`)
        *   Currency Unit (`VBAK-WAERK`)
        *   Quantity (`VBAP-KWMENG`)
        *   Unit of Measure (`VBAP-VRKME`)
    *   Standard ALV functionalities like sorting, filtering, and exporting will be available.

### 3.3 Enhancement Processing Logic – Key steps
1.  The report will retrieve the user-provided selection criteria for Sales Document Number, Material Number, and the 'Top N Rows' limit from the selection screen.
2.  It will then query the SAP database, primarily accessing tables `VBAK` (Sales Document Header) and `VBAP` (Sales Document Item), to fetch relevant sales order item data that matches the specified Sales Document Number and Material Number filters.
3.  **Data Sorting for 'Top N' Logic:** Before applying the 'Top N' limit, the retrieved sales order item records will be sorted based on the following multi-level criteria to ensure consistent, business-relevant results for the "top" records:
    *   **a. Net Value (VBAP-NETWR):** Records will be sorted in **descending order** (items with higher net value will appear first).
    *   **b. Sales Document Number (VBAK-VBELN):** In case of identical Net Values, records will be further sorted by Sales Document Number in **ascending order** (for consistent tie-breaking).
    *   **c. Sales Item Number (VBAP-POSNR):** If both Net Value and Sales Document Number are identical, records will be sorted by Sales Item Number in **ascending order** (for final tie-breaking within a sales document).
4.  After sorting, the report will select and display only the first 'N' records from the sorted dataset, corresponding to the 'Top N Rows' limit specified by the user. If the total number of records matching the criteria is less than 'N', all matching records will be displayed.
5.  The selected data will be presented in an ALV grid format on the output screen.

### 3.4 Table Data Access
The report will access data from the following SAP tables and fields:
*   **VBAK (Sales Document: Header Data):**
    *   `VBELN` (Sales Document Number) - for joining and selection.
    *   `WAERK` (SD Document Currency) - for displaying currency unit.
*   **VBAP (Sales Document: Item Data):**
    *   `VBELN` (Sales Document Number) - for joining with VBAK.
    *   `POSNR` (Sales Document Item)
    *   `MATNR` (Material Number)
    *   `NETWR` (Net Value of the Item)
    *   `KWMENG` (Sales Quantity)
    *   `VRKME` (Sales Unit)

## 4. Error Handling
*   **No Data Found:** If no records match the selection criteria, a clear message will be displayed to the user (e.g., "No sales order item data found for the specified criteria.").
*   **Invalid Input:** Input fields will be validated for correct data types (e.g., 'Top N Rows' must be numeric). Standard SAP value helps (F4) and input validation (e.g., for existing Sales Document or Material Numbers) will be active. Error messages will guide the user to correct invalid entries.
*   **System Errors:** Standard ABAP error handling will be implemented to catch and log any unexpected system errors, providing technical details if required for debugging, while displaying a user-friendly message.

## 5. Level 0 Test Requirements
*   Verify report execution with all selection parameters empty (should display 'Top N' records based on global sort).
*   Test with specific Sales Document Number(s) and Material Number(s) to ensure accurate filtering.
*   Test with a 'Top N Rows' limit (e.g., 10, 50, 1000) to confirm only the specified number of records are displayed and the sorting logic for 'Top N' is correctly applied (Net Value Descending, then Sales Doc Ascending, then Item Ascending).
*   Test scenarios where no data is found for the given criteria.
*   Verify output columns include Sales Document, Item, Material, Net Value, Currency Unit, Quantity, and Unit of Measure.
*   Confirm ALV functionalities (sorting, filtering, export) are working as expected.
*   Test input validation for Sales Document Number and Material Number with invalid/non-existent entries.

## 6. Additional Information
This report is intended for quick, operational lookups and not for complex analytical reporting that might require aggregation or advanced drill-down capabilities. Future enhancements for more sophisticated analysis would require a separate functional specification.

## 7. Acronyms, Abbreviations, and Terms
*   **ABAP:** Advanced Business Application Programming
*   **ALV:** SAP List Viewer
*   **BSA:** Business Systems Analyst
*   **FSD:** Functional Specification Document
*   **MATNR:** Material Number (Unique identifier for a product or service)
*   **NETWR:** Net Value (The value of a sales document item excluding taxes, discounts, etc.)
*   **POSNR:** Sales Document Item Number (Unique line item number within a sales document)
*   **SAP:** Systems, Applications, and Products in Data Processing
*   **UoM:** Unit of Measure
*   **VBAK:** Sales Document: Header Data (SAP table)
*   **VBAP:** Sales Document: Item Data (SAP table)
*   **VBELN:** Sales Document Number (Unique identifier for a sales order or other sales document)
*   **WAERK:** SD Document Currency (Currency key for sales documents)

## 8. References
*   Technical Specification Document for Z_MCCAIN_SALES_ANALYSIS (to be linked upon finalization)
*   SAP Standard Documentation for Sales Order Processing (SD module)

## 9. Document Attributes
*   **Document Name:** Functional Specification - Z_MCCAIN_SALES_ANALYSIS
*   **Version:** 1.1
*   **Date:** 2023-10-27
*   **Author:** Manager Agent / Business Systems Analyst
*   **Status:** Final

---

# Technical Specification Document: Z_MCCAIN_SALES_ANALYSIS

## 1. Enhancement Narrative

### 1.1 Request Details
This document outlines the technical design and specifications for a new custom SAP ABAP report, `Z_MCCAIN_SALES_ANALYSIS`. The report is designed to provide business users with a quick and focused overview of sales document item data, enabling efficient analysis based on specific sales and material criteria, including quantity and unit of measure.

### 1.2 Enhancement Definition
The `Z_MCCAIN_SALES_ANALYSIS` report, officially titled **"McCain Sales Order Item Analysis Report"**, will enable users to retrieve and display key sales item details, including sales document number, item number, material number, net value, currency unit, quantity, and unit of measure. Users can filter the data by sales document number and material number, and also limit the number of results displayed based on a defined sorting order. This report aims to provide immediate insights into sales transactions at the item level via an ALV grid display.

**Program Details:**
*   **Program ID:** `Z_MCCAIN_SALES_ANALYSIS`
*   **Program Title:** `McCain Sales Order Item Analysis Report` (Set in program attributes via `REPORT ... TITLE 'McCain Sales Order Item Analysis Report'`).
*   **Technical Overview:**
    *   **Program Type:** Executable Program (`REPORT`).
    *   **Purpose:** This report is designed to retrieve, sort, and display sales document item data, including quantity and unit of measure, based on user-defined selection criteria for sales document number, material number, and a 'Top N Rows' limit.
    *   **Architecture:** The program utilizes standard ABAP `SELECT` statements for data retrieval and leverages the SAP List Viewer (ALV) functionality for a user-friendly and interactive data display. It primarily interacts with standard SAP transparent tables (`VBAK`, `VBAP`, `MARA`). Data retrieval, sorting, and ALV display logic will be encapsulated in appropriate modularization units.

## 2. Enhancement Functional Requirements

### 2.1 Business Requirements

#### 2.1.1 Objective
The primary objective of the "McCain Sales Order Item Analysis Report" is to support quick lookup and focused review of sales item details, including quantity and unit of measure, for specific sales documents or materials. This report is intended to provide immediate, actionable insights into sales item performance without requiring extensive aggregation or drill-down capabilities in this initial release.

#### 2.1.2 Scope
This report focuses exclusively on sales document item data. It retrieves information from active sales orders based on user-defined selection criteria. The scope includes:
*   Retrieval of Sales Document Number, Sales Document Item Number, Material Number, Net Value, Currency Unit, Quantity, and Unit of Measure.
*   Filtering capabilities by Sales Document Number and Material Number.
*   Limiting the output to a specified number of top records based on a defined sort order.

### 2.2 As-Is/To-Be Functionality
*   **As-Is:** Currently, users may need to navigate multiple standard SAP transactions (e.g., VA03 - Display Sales Order) or combine data from various standard reports to get a consolidated view of sales item data. This can be time-consuming and may not offer the desired level of focused filtering.
*   **To-Be:** The `Z_MCCAIN_SALES_ANALYSIS` report will provide a streamlined, single-point access to sales item data with user-friendly filtering options and an ALV display, significantly reducing the effort required for basic sales item review.

#### 2.2.1 Special Requirements
*   **Top N Rows Display with Defined Order:** The report must include an option for users to limit the number of sales item records displayed in the output. This is crucial for performance and for users who only need to see the top few results. The default limit will be 100 rows, but users can adjust this. Crucially, the "top" records will be determined by a specific sorting order: Net Value (descending), then Sales Document Number (ascending), then Sales Item Number (ascending).

### 2.3 SAP Standard Reference
This is a custom report (`Z` development) designed to meet specific business reporting needs. It leverages standard SAP sales data structures and tables, specifically drawing information from the Sales Document Item Data (VBAP) and Sales Document Header Data (VBAK) tables, and referencing General Material Data (MARA) for input validation. The output will utilize standard ALV (SAP List Viewer) functionality.

### 2.4 Inputs and Dependencies
The report requires the following user inputs on its selection screen:

*   **Sales Document Number (S_VBELN):**
    *   **Purpose:** Allows users to filter the report results by one or more specific sales order numbers.
    *   **Functionality:** Users can enter a single sales document number, a range of numbers, or select multiple individual numbers.
    *   **Dependency:** This field directly filters data from the Sales Document Item data (`VBAP`). Input validation will be active against `VBAK-VBELN`.

*   **Material Number (S_MATNR):**
    *   **Purpose:** Enables users to filter the report results by one or more specific material numbers.
    *   **Functionality:** Users can enter a single material number, a range of numbers, or select multiple individual numbers.
    *   **Dependency:** This field directly filters data from the Sales Document Item data (`VBAP`). Input validation will be active against `MARA-MATNR`.

*   **Top N Rows (P_TOP):**
    *   **Purpose:** Controls the maximum number of sales item records that will be displayed in the report output after sorting.
    *   **Functionality:** Users can enter any whole positive number. The default value will be 100. If left blank or set to 0, the report will display all records matching other criteria (up to system limits).
    *   **Dependency:** This parameter limits the data retrieval from the database after sorting.

### 2.5 Regulatory / Compliance Requirements
No specific regulatory or compliance requirements have been identified for this report. The report provides read-only access to existing sales data and does not modify any system records.

### 2.6 Assumptions
*   It is assumed that the underlying sales order data in SAP is accurate and up-to-date.
*   Users will have the necessary authorizations to execute the report and access the relevant sales data.
*   The report's performance will be acceptable for typical data volumes, especially with the "Top N Rows" functionality and optimized `SELECT` statements.

## 3. Enhancement Specifications

### 3.1 User Exit /BADI/Routines/ Details
This report is a standalone custom ABAP program and does not utilize standard SAP user exits or Business Add-Ins (BADIs). Its core logic is encapsulated within custom subroutines or methods for data retrieval, processing (sorting), and ALV display.

### 3.2 Transaction Screens

The report will have a single selection screen (initial screen) where users enter their filtering criteria. After execution, the results will be displayed in an interactive ALV grid format.

#### 3.2.1 Screen Flow
1.  User executes the report transaction code (to be assigned).
2.  The selection screen is presented, showing the "Sales Document Number", "Material Number", and "Top N Rows" input fields.
3.  User enters desired criteria and executes the report (e.g., by pressing F8 or the Execute button).
4.  The report output is displayed in a new screen as an ALV grid.

**Selection Screen Objects:**
The report defines the following input fields on its selection screen:

| Technical Name | Type           | Associated ABAP Dictionary Data Element / Domain | Properties                                   |
| :------------- | :------------- | :----------------------------------------------- | :------------------------------------------- |
| `S_VBELN`      | `SELECT-OPTIONS` | `VBELN_VA` (from `VBAK-VBELN`)                   | Standard `SELECT-OPTIONS` properties (range, multiple selection), input validation active. |
| `S_MATNR`      | `SELECT-OPTIONS` | `MATNR` (from `MARA-MATNR`)                      | Standard `SELECT-OPTIONS` properties (range, multiple selection), input validation active. |
| `P_TOP`        | `PARAMETERS`   | `INT4` (Standard ABAP type for integer)          | `TYPE i`, `DEFAULT 100`, `NO-ZERO` (ensures positive integer) |

### 3.3 Enhancement Processing Logic – Key steps

The program execution follows a straightforward sequential flow:

*   **Program Entry Point:** The program execution begins at `START-OF-SELECTION`.
*   **Overall Logic Sequence:**
    1.  Initialization (e.g., build field catalog for ALV).
    2.  Call a subroutine/method to `get_sales_data`.
    3.  Call a subroutine/method to `display_sales_data_alv`.

#### 3.3.1 `FORM get_sales_data` (or equivalent method)

*   **Purpose:** This subroutine/method is responsible for retrieving sales document item and header data from the database based on the user's selection criteria, and then sorting it.
*   **Logic:**
    *   A `SELECT` statement with `INNER JOIN` is executed to fetch specific fields from the `VBAP` and `VBAK` tables.
    *   The `INTO CORRESPONDING FIELDS OF TABLE gt_sales_data` clause ensures that the selected fields (`vbeln`, `posnr`, `matnr`, `netwr`, `kwmeng`, `vrkme`, `waerk`) are populated into the corresponding components of the internal table `gt_sales_data`.
    *   **Data Source:** `VBAP` (Sales Document Item Data) `INNER JOIN` `VBAK` (Sales Document Header Data).
    *   **`WHERE` Conditions:**
        *   `VBAP~vbeln IN s_vbeln`: Filters sales document numbers based on the input provided in the `S_VBELN` select-option.
        *   `VBAP~matnr IN s_matnr`: Filters material numbers based on the input provided in the `S_MATNR` select-option.
    *   **Sorting:** After data retrieval, the internal table `gt_sales_data` will be explicitly sorted before the `UP TO` clause is applied (if done in ABAP layer, otherwise `ORDER BY` in `SELECT`):
        `SORT gt_sales_data BY netwr DESCENDING vbeln ASCENDING posnr ASCENDING.`
    *   **Limiting Clause:** `UP TO p_top ROWS` (if sorting done in DB) or `DELETE ADJACENT DUPLICATES` after `SORT` (if sorting done in ABAP and then `p_top` records are taken from sorted table): The number of records retrieved/processed will be limited to the value specified in the `P_TOP` parameter.
    *   **Data Manipulation/Preparation:** No explicit data manipulation or calculations are performed within this form/method beyond data retrieval and sorting.

#### 3.3.2 `FORM display_sales_data_alv` (or equivalent method)

*   **Purpose:** This subroutine/method is responsible for displaying the sorted sales data using ALV functionality.
*   **Logic:**
    *   If `gt_sales_data` is not empty, populate an ALV field catalog to define column properties (e.g., headings, data types, output length, reference fields for currency/quantity).
    *   Call the appropriate ALV function module (e.g., `REUSE_ALV_GRID_DISPLAY` or `CL_SALV_TABLE`) to display the `gt_sales_data` internal table.
    *   **Error Handling (No Data):** If `gt_sales_data` is empty, a message indicating "No data found" will be displayed.

### 3.4 Table Data Access

The report primarily accesses data from the **Sales Document Item Data table (VBAP)** and **Sales Document Header Data table (VBAK)**. It uses the Sales Document Header Data (VBAK) and General Material Data (MARA) tables for validating input values provided on the selection screen, ensuring that users enter valid sales document and material numbers.

**Key Global Data Objects and Structures:**

| Object Name       | Type                     | Description                                            | Associated DDIC Object(s) / Inferred Structure |
| :---------------- | :----------------------- | :----------------------------------------------------- | :--------------------------------------------- |
| `VBAK`            | `TABLES` (Work Area)     | Sales Document: Header Data (Implicit declaration)     | `VBAK` (Database Table)                        |
| `VBAP`            | `TABLES` (Work Area)     | Sales Document: Item Data (Implicit declaration)       | `VBAP` (Database Table)                        |
| `MARA`            | `TABLES` (Work Area)     | General Material Data (Implicit declaration)           | `MARA` (Database Table)                        |
| `gt_sales_data`   | `TABLE OF ty_sales`      | Internal table to store retrieved and sorted sales item data. | Inferred structure `ty_sales` (see below)      |
| `gs_sales_data`   | `ty_sales`               | Work area for processing individual records from `gt_sales_data`. | Inferred structure `ty_sales` (see below)      |
| `gt_fieldcat`     | `LVC_T_FCAT`             | Internal table for ALV field catalog.                  | `LVC_S_FCAT` (Structure for ALV field catalog) |

**Inferred Structure `ty_sales`:**
The structure `ty_sales` will be explicitly defined or implicitly determined by the `SELECT` statement and will contain the following components:

| Component Name | Description                 | DDIC Reference (from VBAP/VBAK) | Data Element |
| :------------- | :-------------------------- | :------------------------------ | :----------- |
| `VBELN`        | Sales Document Number       | `VBAP-VBELN` / `VBAK-VBELN`     | `VBELN_VA`   |
| `POSNR`        | Sales Document Item Number  | `VBAP-POSNR`                    | `POSNR`      |
| `MATNR`        | Material Number             | `VBAP-MATNR`                    | `MATNR`      |
| `NETWR`        | Net Value of the Sales Item | `VBAP-NETWR`                    | `NETWR_AP`   |
| `WAERK`        | Document Currency           | `VBAK-WAERK`                    | `WAERK`      |
| `KWMENG`       | Sales Quantity              | `VBAP-KWMENG`                   | `KWMENG`     |
| `VRKME`        | Sales Unit                  | `VBAP-VRKME`                    | `VRKME`      |

### Output Details

*   **Display Method:** The final output is presented as an **interactive ALV grid report** using standard ALV function modules (e.g., `REUSE_ALV_GRID_DISPLAY` or `CL_SALV_TABLE`).
*   **Output Structure and Fields:**
    *   Each row in the ALV output corresponds to a single sales document item record from the `gt_sales_data` internal table.
    *   **Column Headers:** Clear, user-friendly column headers will be defined in the ALV field catalog (e.g., "Sales Document", "Item No.", "Material", "Net Value", "Currency", "Quantity", "Unit of Measure").
    *   **Field Mapping and Formatting:**
        *   `VBELN`: Sales Document Number
        *   `POSNR`: Sales Document Item Number
        *   `MATNR`: Material Number
        *   `NETWR`: Net Value (formatted as currency with 2 decimal places, linked to `WAERK` for currency symbol/precision).
        *   `WAERK`: Currency Unit (displayed alongside `NETWR`).
        *   `KWMENG`: Sales Quantity (formatted based on unit of measure).
        *   `VRKME`: Sales Unit (displayed alongside `KWMENG`).
    *   **ALV Functionality:** Users will have access to standard ALV features such as sorting, filtering, aggregation (if enabled for numeric fields), layout saving, and export to various formats (e.g., Excel).

## 4. Error Handling
*   **Invalid Input:** Standard SAP input validation will be active for `S_VBELN` (against `VBAK`) and `S_MATNR` (against `MARA`). Standard SAP error messages will be displayed for invalid entries on the selection screen. The `P_TOP` parameter will ensure a positive integer.
*   **No Data Found:** If no data matches the selection criteria, the ALV will display a message indicating that no records were found (e.g., "No sales order item data found for the specified criteria.").
*   **System Errors:** Standard ABAP runtime errors will be handled by the system. Robust error handling (e.g., `TRY...CATCH` blocks for critical operations) should be implemented to ensure graceful termination and provide informative messages to the user and technical logs.

## 5. Level 0 Test Requirements
The following basic test scenarios should be performed:
*   Execute the report with no selection criteria (expecting default "Top 100" records sorted by Net Value Descending, then Sales Doc Ascending, then Item Ascending).
*   Test with specific Sales Document Number(s) and Material Number(s) to ensure accurate filtering.
*   Test with a 'Top N Rows' limit (e.g., 10, 50, 1000) to confirm only the specified number of records are displayed and the sorting logic for 'Top N' is correctly applied (Net Value Descending, then Sales Doc Ascending, then Item Ascending).
*   Test scenarios where no data is found for the given criteria.
*   Verify all new output fields (Currency Unit, Quantity, Unit of Measure) are displayed correctly.
*   Confirm ALV functionalities (sorting, filtering, export, layout saving) are working as expected.
*   Test input validation for Sales Document Number and Material Number with invalid/non-existent entries.

## 6. Additional Information
*   This report is intended for quick analysis and focused review of sales item details. It does not include advanced features like drill-down into related documents (beyond standard ALV double-click capabilities), complex aggregations, or specific business analytics functions beyond what ALV offers.
*   Future enhancements could include additional filtering options, output fields, or integration with analytical tools as business needs evolve.

## 7. Acronyms, Abbreviations, and Terms

*   **ABAP:** Advanced Business Application Programming (SAP's proprietary programming language)
*   **ALV:** SAP List Viewer (standard SAP component for displaying interactive lists)
*   **FSD:** Functional Specification Document
*   **MATNR:** Material Number (Unique identifier for a product or service)
*   **NETWR:** Net Value (The value of a sales document item excluding taxes, discounts, etc.)
*   **POSNR:** Sales Document Item Number (Unique line item number within a sales document)
*   **SAP:** Systems, Applications & Products in Data Processing (ERP software)
*   **TSD:** Technical Specification Document
*   **UoM:** Unit of Measure
*   **VBAK:** Sales Document Header Data (SAP table)
*   **VBAP:** Sales Document Item Data (SAP table)
*   **VBELN:** Sales Document Number (Unique identifier for a sales order or other sales document)
*   **WAERK:** SD Document Currency (Currency key for sales documents)

## 8. References
*   Functional Specification Document for Z_MCCAIN_SALES_ANALYSIS
*   SAP ALV Documentation
*   SAP ABAP Programming Guidelines

## 9. Document Attributes

*   **Document Name:** Technical Specification for Z_MCCAIN_SALES_ANALYSIS
*   **Version:** 1.1
*   **Date:** 2023-10-27
*   **Author:** Manager Agent / Technical Documentation Specialist
*   **Status:** Final